In [1]:
import pandas as pd
import numpy as np


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Selection

In [2]:
financial_ratios_table = pd.read_csv('../../DATA/MODEL/financial_ratios_df.csv')

In [3]:
column = 'SAL'

pivot = pd.pivot_table(financial_ratios_table.query('year <= 2019'), index = 'entity_ico', columns='year', values=column)
bool_wide = (pivot == pivot)
bool_wide['count'] = bool_wide.apply(sum, axis = 1)
bool_wide['count'].value_counts()

bool_wide.reset_index(inplace=True)
bool_wide.columns = ['entity_ico', 'y14', 'y15', 'y16', 'y17', 'y18', 'y19', 'count']
bool_wide

,entity_ico,y14,y15,y16,y17,y18,y19,count
0,671.0,True,True,True,True,False,False,4
1,698.0,True,True,True,True,True,True,6
2,710.0,True,True,True,True,True,True,6
3,2313.0,True,True,True,True,True,True,6
4,5819.0,True,True,True,True,True,True,6
...,...,...,...,...,...,...,...,...
227369,52831604.0,False,False,False,False,False,True,1
227370,52831728.0,False,False,False,False,False,True,1
227371,52834476.0,False,False,False,False,False,True,1
227372,60748940.0,True,True,False,False,False,False,2


In [4]:
entity_ico_list = list(bool_wide.query('y15 and y16 and y17')['entity_ico'].unique())

financial_ratios_table.query('entity_ico in @entity_ico_list', inplace= True)
# financial_ratios_table

## Outliers

In [5]:

FINANCIAL_RATIOS = [
    'L3', 'L2', 'L1', 'CF_CL', 'CASH_TA', 'SAL_TA', 'TL_SAL',
    'INV_COST', 'INV_SAL', 'CC_SAL', 'TA_SAL',
    'TL_TA', 'CF_TL', 'CL_TA', 'NCL_TA', 'EQ_TL',
    'EQ_TA', 
    # 'EBIT_INT', 
    'CL_CC', 'ROE', 'EAT_TA',
    'ROA_BRUTTO', 'CF_TA', 'CF_SAL', 'ROS',
    'EAT_YIE', 'ROI', 'ROA_NETTO']

SKNACE_DIVISIONS = list(financial_ratios_table['sknace_division'].unique())
YEARS = list(financial_ratios_table['year'].unique())
TEMPLATES = list(financial_ratios_table['financial_report_template_id'].unique())

In [6]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

def get_outliers(l):
    q1 = l.quantile(0.1)  
    q3 = l.quantile(0.9)
    iqr = q3-q1
    fenceLow = q1 - 1.5 * iqr
    fenceHigh = q3 + 1.5 * iqr

    return [(i<fenceLow or i>fenceHigh) for i in l]


# def get_outliers(l, z_threshold=0.5):
#     mean = np.mean(l)
#     std_dev = np.std(l)
#     z_scores = [(i - mean) / std_dev for i in l]
#     return [abs(z) > z_threshold for z in z_scores]


def plot_histograms(df, columns):
    # Plotting the histograms
    fig, axes = plt.subplots(nrows=int(len(columns) / 3) + 1, ncols=3, figsize=(20, 30))
    axes = axes.flatten()

    for i, column in enumerate(columns):
        ax = axes[i]

        data = df[column]
        
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        
        IQR = Q3 - Q1
        
        # lower_bound = Q1 - 1.5 * IQR
        # upper_bound = Q3 + 1.5 * IQR

        # cleaned_data = data[~((data < lower_bound) | (data > upper_bound))]

        bin_width = 2 * IQR / (len(data) ** (1/3))

        bin_width = bin_width if bin_width > 0 else 1
        num_bins = int(np.ceil((np.max(data) - np.min(data)) / bin_width))

        ax.hist(data, bins=num_bins, color='skyblue', edgecolor='black')
        ax.set_title(column)
        ax.set_xlabel('Value')
        ax.set_ylabel('Frequency')

        

    plt.tight_layout()
    plt.show()


In [ ]:
display(financial_ratios_table)

In [41]:
outlier_boolean = pd.DataFrame()

for year in YEARS:
    print(year)
    for sknace in SKNACE_DIVISIONS:
        for template_id in TEMPLATES:
            subset = financial_ratios_table.query('financial_report_template_id == @template_id and year == @year and sknace_division == @sknace')[['financial_statement_id'] + FINANCIAL_RATIOS].set_index(['financial_statement_id'])
            outliers = subset.apply(get_outliers)

            # print('size', subset.shape[0])
            # print('not outliers', round(subset[~outliers.apply(lambda x:any(x), axis=1)].shape[0]/subset.shape[0], 2))
            # print('outliers', round(subset[outliers.apply(lambda x:any(x), axis=1)].shape[0]/subset.shape[0], 2))

            # display(outliers.apply(lambda x: x.value_counts(normalize=True)).T.sort_values(True, ascending = False))

            non_outliers_index = subset[~outliers.apply(lambda x:any(x), axis=1)].index
            outliers_index = subset[outliers.apply(lambda x:any(x), axis=1)].index



            subset_non_outliers = pd.DataFrame([0 for _ in range(len(non_outliers_index))], index = non_outliers_index, columns = ['is_outlier'])
            subset_outliers = pd.DataFrame([1 for _ in range(len(outliers_index))], index = outliers_index, columns = ['is_outlier'])


            outlier_boolean = pd.concat([outlier_boolean, subset_non_outliers, subset_outliers], axis = 0)

   

2014
2015
2016
2017
2018
2019
2020
2021


In [45]:
display(financial_ratios_table.shape)

financial_ratios_table = financial_ratios_table.merge(outlier_boolean.reset_index(), on = 'financial_statement_id', how = 'left')

display(financial_ratios_table.shape)
display(financial_ratios_table)

(934262, 82)

(934262, 83)

,financial_report_id,financial_statement_id,financial_report_order_number,entity_id,approved_date,entity_ico,year,month,financial_report_template_id,entity_name,establishment_date,sknace_code,entity_state,CA,CASH,CL,EQ,NCL,TA,TL,CC,INV,CF_NETTO,CF_SELFFIN,EAT,EBIT,SAL,COST,INT,YIE,L3,L2,L1,CF_CL,CASH_TA,SAL_TA,TL_SAL,INV_COST,INV_SAL,CC_SAL,TA_SAL,TL_TA,CF_TL,CL_TA,NCL_TA,EQ_TL,EQ_TA,EBIT_INT,CL_CC,ROE,EAT_TA,ROA_BRUTTO,CF_TA,CF_SAL,ROS,EAT_YIE,ROI,ROA_NETTO,DPHZ_vat_registration_cancelled,DPHZ_vat_registration_cancelled_1y_off,DPHZ_vat_registration_cancelled_2y_off,RO_cancelled,RO_cancelled_1y_off,RO_cancelled_2y_off,RUZ_cancelled,RUZ_cancelled_1y_off,RUZ_cancelled_2y_off,RUZ_established,RUZ_established_1y_off,RUZ_established_2y_off,RU_konkurz,RU_konkurz_1y_off,RU_konkurz_2y_off,RU_ostatne_konania,RU_ostatne_konania_1y_off,RU_ostatne_konania_2y_off,RUZ_indirect_criterion,RUZ_indirect_criterion_1y_off,RUZ_indirect_criterion_2y_off,sknace_division_name,sknace_division,sknace_subcategory,is_outlier
0,4890214,2575566,2,25527,2015-07-31,671.0,2014,1,699.0,"Kerametal, akciová spoločnosť,",1970-02-02,46180.0,NaN,5174843.0,1479.0,1366130.0,3807863.0,1255.0,5175248.0,1367385.0,5173364.0,0.0,-547786.0,-548266.0,-550029.0,-549549.0,6350.0,560563.0,0.0,11014.0,3.787958,3.787958,0.001083,-0.400976,0.000286,0.001227,215.336220,0.000000,0.000000,814.702992,293399.886614,0.264216,-0.400608,0.263974,0.000243,2.784778,0.735784,NaN,0.264070,-0.144446,-0.106281,-0.106188,-0.105847,-86.265512,-86.618740,-49.939078,-0.106281,-0.106281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,1.0,"Veľkoobchod, okrem motorových vozidiel a motoc...",G,46.0,1
1,5392251,2976863,2,25527,2016-12-31,671.0,2015,1,699.0,"Kerametal, akciová spoločnosť, Bratislava",1970-02-02,46180.0,NaN,5174245.0,1868.0,1693023.0,3480567.0,1255.0,5174845.0,1694278.0,5172377.0,0.0,-326822.0,-327302.0,-327302.0,-326822.0,3633.0,336975.0,0.0,10153.0,3.056217,3.056217,0.001103,-0.193040,0.000361,0.000702,466.357831,0.000000,0.000000,1423.720617,512783.980182,0.327407,-0.192898,0.327164,0.000243,2.054307,0.672593,NaN,0.327320,-0.094037,-0.063249,-0.063156,-0.063156,-89.959262,-90.091385,-32.236974,-0.063249,-0.063249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1.0,"Veľkoobchod, okrem motorových vozidiel a motoc...",G,46.0,1
2,5887438,3375702,2,25527,2017-12-27,671.0,2016,1,699.0,"Kerametal, akciová spoločnosť, Bratislava",1970-02-02,46180.0,NaN,14679.0,1034.0,1697982.0,-1683081.0,0.0,14901.0,1697982.0,13645.0,0.0,-5163168.0,-5163648.0,-5163648.0,-5163168.0,1994.0,5166660.0,0.0,3492.0,0.008645,0.008645,0.000609,-3.040767,0.069391,0.133817,851.545637,0.000000,0.000000,6.843029,2690.250752,113.950876,-3.040767,113.950876,0.000000,-0.991224,-112.950876,NaN,124.439868,NaN,-346.530300,-346.498087,-346.498087,-2589.352056,-2589.592778,-1478.707904,-346.530300,-346.530300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,1.0,1.0,"Veľkoobchod, okrem motorových vozidiel a motoc...",G,46.0,1
3,6361460,3757740,1,25527,2018-12-17,671.0,2017,1,699.0,"Kerametal, akciová spoločnosť, Bratislava",1970-02-02,46180.0,NaN,14501.0,942.0,1729112.0,-1714376.0,0.0,14736.0,1729112.0,13559.0,0.0,-30827.0,-31307.0,-31307.0,-30827.0,1993.0,33542.0,11844.0,2715.0,0.008386,0.008386,0.000545,-0.017828,0.063925,0.135247,867.592574,0.000000,0.000000,6.803312,2661.796287,117.339305,-0.017828,117.339305,0.000000,-0.991478,-116.339305,-1.602752,127.525039,NaN,-2.124525,-2.091952,-2.091952,-15.467637,-15.708480,-11.531123,-1.320779,-2.124525,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,1.0,1.0,"Veľkoobchod, okrem motorových vozidiel a motoc...",G,46.0,1
4,4593745,2340588,1,16410,2015-06-11,698.0,2014,1,699.0,"TECHNOPOL, a.s.",1970-02-18,70220.0,NaN,1543495.0,315617.0,3796505.0,17186564.0,102542.0,21085611.0,3899047.0,1227526.0,352.0,-245840.0,-246827.0,-275229.0,-274242.0,282838.0,611268.0,51413.0,337026.0,0.406

In [46]:
financial_ratios_table.to_csv('../../DATA/MODEL/FINAL_financial_ratios_df.csv', index = False)